## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

## Загрузка данных

Импорт библиотек

In [1]:
import pandas as pd
import re

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sirena0789/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sirena0789/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sirena0789/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sirena0789/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sirena0789/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Загрузка файла с данными

In [3]:
data = pd.read_csv('/Users/sirena0789/Downloads/toxic_comments.csv', index_col=0)
corpus = data['text']

Знакомство с одним из комментариев

In [4]:
corpus[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

## Подготовка данных

Очистка текста от лишних символов

In [5]:
def clear_text(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

In [6]:
corpus = corpus.apply(clear_text)

Лемматизация текста

In [7]:
lemmatizer = WordNetLemmatizer()
corpus = corpus.apply(lambda sentence: " ".join([lemmatizer.lemmatize(w,"n") for w in nltk.word_tokenize(sentence)]))

In [8]:
corpus

0         explanation why the edits made under my userna...
1         daww he match this background colour im seemin...
2         hey man im really not trying to edit war it ju...
3         more i cant make any real suggestion on improv...
4         you sir are my hero any chance you remember wh...
                                ...                        
159446    and for the second time of asking when your vi...
159447    you should be ashamed of yourself that is a ho...
159448    spitzer umm there no actual article for prosti...
159449    and it look like it wa actually you who put on...
159450    and i really dont think you understand i came ...
Name: text, Length: 159292, dtype: object

Разделение выборок в соотношении 50/50

In [9]:
target = data['toxic'].values
features = corpus
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.5,
                                                                            random_state=12, stratify=target)

features_train.shape[0], features_test.shape[0]

(79646, 79646)

Загрузка стоп слов на английском языке

In [11]:
stopwords = set(nltk_stopwords.words('english'))

Векторизация текста

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train) 
tf_idf_test = count_tf_idf.transform(features_test) 

**Вывод:**  
Данные подготовлены к обучению моделей.

## Обучение моделей

Модель LogisticRegression

In [16]:
%%time


pipe = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
                 ('model',LogisticRegression(random_state = 12, solver='liblinear', max_iter=200))])

param = [{'model__penalty' : ['l1', 'l2'], 'model__C': list(range(1,15,3))}]
grid = GridSearchCV(pipe, param_grid=param, scoring='f1', cv=3, verbose=True, n_jobs=-1)
model = grid.fit(features_train, target_train)
best_model = model.best_estimator_
print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters: {'model__C': 4, 'model__penalty': 'l1'}
Best score: 0.7658667158796392
CPU times: user 3.55 s, sys: 1.26 s, total: 4.82 s
Wall time: 22.5 s


Модель RandomForestClassifier

In [17]:
%%time

pipe = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
                 ('model',RandomForestClassifier(random_state = 12))])
param = {'model__n_estimators': list(range(50,300,50)), 'model__max_depth':[5,15], 'model__max_features':['sqrt', 'log2']}
grid = GridSearchCV(pipe, param_grid=param, scoring='f1', cv=3, verbose=True, n_jobs=-1)
model = grid.fit(features_train, target_train)
print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best parameters: {'model__max_depth': 15, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best score: 0.000740832224097631
CPU times: user 5.26 s, sys: 2.57 s, total: 7.83 s
Wall time: 53.7 s


Модель LinearSVC

In [18]:
%%time

pipe = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
                 ('model', LinearSVC(random_state=12, max_iter=1000))])
param = [{'model__penalty' : ['l1', 'l2'], 'model__C': list(range(1,15,3))}]
grid = GridSearchCV(pipe, param_grid=param, scoring='f1', cv=3, verbose=True, n_jobs=-1)
model = grid.fit(features_train, target_train)
print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters: {'model__C': 1, 'model__penalty': 'l2'}
Best score: 0.7585554063794299
CPU times: user 3.32 s, sys: 1.2 s, total: 4.52 s
Wall time: 17.8 s


Сведем результаты в таблицу

In [19]:
index = ['LogisticRegression',
         'RandomForestClassifier',
         'LinearSVC']
data = {'F1 на обучающей выборке':[0.765878, 0.0, 0.758211],
        'Время обучения на обучающей выборке, сек':[154, 52.8, 17.9]}

scores_data = pd.DataFrame(data=data, index=index)
scores_data

,F1 на обучающей выборке,"Время обучения на обучающей выборке, сек"
LogisticRegression,0.765878,154.0
RandomForestClassifier,0.000000,52.8
LinearSVC,0.758211,17.9


**Вывод:**  
Лучшей моделью на этапе обучения является:
- Модель: LogisticRegression;  
- Гиперпараметры: C = 4, penalty = l1, random_state=12, solver='liblinear', max_iter=200;     
- F1 на обучающей выборке: 0.766028.  

## Тестирование лучшей модели

In [20]:
%%time

pred = best_model.predict(features_test)

CPU times: user 1.87 s, sys: 25.5 ms, total: 1.9 s
Wall time: 1.92 s


In [21]:
f1_score(target_test, pred)

0.7741628096923496

##  Вывод

    Задача проекта состояла в поиске "токсичных" комментариев из предлагаемого набора данных.  
    В работе было проведено знакомство с данными, очистка текста от лишних символов, текст был привден к нижнему регистру.     
    Далее была проведена лемматизация текста с использованием WordNetLemmatizer(), данные разделились на тестовую и обучающую выборки.
    Затем были обучены три модели с подбором гиперпараметров с помощью GridSearchCV и Pipeline, который также проводит векторизацию кросс валидацию данных.  
    По итогам обучения в лидеры вышли модели LogisticRegression с F1 0.765878 и LinearSVC с F1 0.758211.  
    Отталкиваясь от запрашеваемой метрики тестированию подверглась модель LogisticRegression.  
    Метрика качества F1 на тестировании прошла запрашеваемый порог.

 Итоговая модель:
- Модель: LogisticRegression;  
- Гиперпараметры: C = 4, penalty = l1, random_state=12, solver='liblinear', max_iter=200;     
- F1 на тестовой выборке: 0.774799;
- Время предсказания модели, сек: 2.03.